# NYC Museum Map Blog

### Scrape data to get the full descriptions of each museum

In [5]:
# import beautifulsoup and set up link to scrape
import csv
import requests
import urllib.request
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/List_of_museums_in_New_York_City'
response = requests.get(url)
html = response.content

soup = BeautifulSoup(html, 'html.parser')

In [6]:
# metaphorically "throw net over" the content that we want.
rows = soup.findAll('tr')

In [7]:
# narrow down the details to get what we want
name = rows[4].td.text.strip('\n')
name

'AIGA National Design Center'

In [8]:
# getting "focus" or category of museum
focus = rows[4].td.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.text.strip('\n').title()
focus

'Design, Decorative Arts, Architecture'

In [9]:
# getting "summary" or description of museum
summary = rows[4].td.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.text.strip('\n').title()
summary

'Website, Public Gallery Of The Aiga Dedicated To Presenting Examples Of Outstanding Contemporary Design'

In [10]:
#show progress bar when we scrape. use this!
from tqdm import tqdm 

### <i>After using code below, comment out to avoid running webscraper twice

### ScrapeLoop through Wikipedia with codes we wrote above, save data to CSV

In [12]:
# file = open('nycmuseumdata.csv', 'w')
# writer = csv.writer(file)

# writer.writerow(['name', 'focus', 'summary'])

# for i in tqdm(range(2, len(rows))):
#     for row in rows:
#         try:
#             name = rows[i].td.text.strip('\n')
#             focus = rows[i].td.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.text.strip('\n').capitalize()
#             summary = rows[i].td.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.text.strip('\n').capitalize()
#         except:
#             pass
#     writer.writerow([name, focus, summary])


# Merge datasets (dataframe(df) with location + df with data)

In [98]:
#successfully scraped museum data with focus and summary details
df_data = pd.read_csv('nycmuseumdata.csv')
df_data.head(10)

,name,focus,summary
0,9/11 Tribute Museum,American and ny history,History of the september 11 attacks and tours ...
1,African Burial Ground National Monument,African and african-american,Visitor center and memorial to an 18th-century...
2,AIGA National Design Center,"Design, decorative arts, architecture","Website, public gallery of the aiga dedicated ..."
3,A.I.R. Gallery,Contemporary art,Contemporary art by female artists
4,Alice Austen House,"Photography, film, new media","Home of photographer alice austen, also featur..."
5,American Academy of Arts and Letters,Contemporary art,Exhibitions of its members in american literat...
6,American Folk Art Museum,Folk art,American folk art
7,American Museum of Natural History,Nature,The world's largest natural history museum. in...
8,American Numismatic Society Museum,Financial,Exhibits of coins and medals from all periods ...
9,Americas Society,"Hispanic, latino, and caribbean",Exhibits of historic and contemporary art from...


In [99]:
# This is the other dataset with coordiantes from NYC gov
file = 'Museum Dataset.xlsx'
df_location = pd.read_excel(file) # plots df with coordinates/location details
df_location.head(3)

,the_geom,NAME,TEL,URL,ADRESS1,ADDRESS2,CITY,ZIP
0,POINT (-74.01375579519738 40.703816216918035),Alexander Hamilton U.S. Custom House,(212) 514-3700,http://www.oldnycustomhouse.gov/,1 Bowling Grn,NaN,New York,10004
1,POINT (-74.06303178855111 40.615120837755356),Alice Austen House Museum,(718) 816-4506,http://www.aliceausten.org/,2 Hylan Blvd,NaN,Staten Island,10305
2,POINT (-73.94729768541572 40.833853500753314),American Academy of Arts and Letters,(212) 368-5900,http://www.artsandletters.org/,633 W. 155th St.,NaN,New York,10032


In [100]:
# let's lowercase our columns and add an underscore between words for easy access
## uneccessary but I like doing this for future convenience
df_location.columns = [x.lower() for x in df.columns] # show columns
df_location.columns = [x.replace(' ', '_').lower() for x in df_location.columns] #lower case and replace spaces with '_' for accessibility
df_location.columns

Index(['the_geom', 'name', 'tel', 'url', 'adress1', 'address2', 'city', 'zip'], dtype='object')

In [101]:
# clean up so that coordinates can be used - remove leading white space, remove the word POINT
df_location['the_geom'] = df_location['the_geom'].apply(lambda x: x.strip('POINT').lstrip().replace(' ', ', '))
df_location['the_geom']

0      (-74.01375579519738, 40.703816216918035)
1      (-74.06303178855111, 40.615120837755356)
2      (-73.94729768541572, 40.833853500753314)
3       (-73.97810302110001, 40.76162497138548)
4        (-74.0396848374904, 40.69905626244391)
                         ...                   
125     (-73.99388579658319, 40.73807723317813)
126     (-73.99379427245329, 40.73795994708654)
127    (-73.99392890421056, 40.738015384101416)
128       (-73.99381737049632, 40.738045683847)
129    (-73.99386877223978, 40.737986764588406)
Name: the_geom, Length: 130, dtype: object

In [115]:
# merge "name" to get these two datasets together. Do an outer join to keep everything
df_final = df_location.merge(df_data, on='name', how='outer')
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306 entries, 0 to 305
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   the_geom  132 non-null    object 
 1   name      306 non-null    object 
 2   tel       130 non-null    object 
 3   url       132 non-null    object 
 4   adress1   132 non-null    object 
 5   address2  15 non-null     object 
 6   city      132 non-null    object 
 7   zip       132 non-null    float64
 8   focus     181 non-null    object 
 9   summary   223 non-null    object 
dtypes: float64(1), object(9)
memory usage: 26.3+ KB


In [121]:
# take df_final to excel to fill in any missing data
## Some of the museums are spelled slightly different and could not be filled in properly. 
### I take it to excel to manually fill them in as there isn't too many.
df_final.to_excel('museum_cleaning.xlsx')

In [125]:
# now that I've filled in as much as I could, + 29 extra data points
df_final = pd.read_excel('museum_cleaning.xlsx').drop(columns='Unnamed: 0')
df_final.head()

,the_geom,name,tel,url,adress1,address2,city,zip,focus,summary
0,NaN,9/11 Tribute Museum,NaN,NaN,NaN,NaN,NaN,NaN,American and ny history,History of the september 11 attacks and tours ...
1,NaN,A.I.R. Gallery,NaN,NaN,NaN,NaN,NaN,NaN,Contemporary art,Contemporary art by female artists
2,NaN,African Burial Ground National Monument,NaN,NaN,NaN,NaN,NaN,NaN,African and african-american,Visitor center and memorial to an 18th-century...
3,NaN,AIGA National Design Center,NaN,NaN,NaN,NaN,NaN,NaN,"Design, decorative arts, architecture","Website, public gallery of the aiga dedicated ..."
4,"(-74.01375579519738, 40.703816216918035)",Alexander Hamilton U.S. Custom House,(212) 514-3700,http://www.oldnycustomhouse.gov/,1 Bowling Grn,NaN,New York,10004.0,NaN,NaN


In [141]:
# so this is how much we have now... 79 full data rows. However, I want more.
df_final[(df_final['the_geom'].notna()) & (df_final['summary'].notna())].reset_index().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   index     79 non-null     int64  
 1   the_geom  79 non-null     object 
 2   name      79 non-null     object 
 3   tel       79 non-null     object 
 4   url       79 non-null     object 
 5   adress1   79 non-null     object 
 6   address2  8 non-null      object 
 7   city      79 non-null     object 
 8   zip       79 non-null     float64
 9   focus     55 non-null     object 
 10  summary   79 non-null     object 
dtypes: float64(1), int64(1), object(9)
memory usage: 6.9+ KB


In [138]:
# query the rows with coordinates and no summary, let's see what else we could do to supplement -
## ...ok since there's 50 more ... that's all folks. Let me know if there's a better way than googling each one!
### we'll plot the 79 that we have now.
df_final[(df_final['the_geom'].notna()) & (df_final['summary'].isna())].reset_index()

,index,the_geom,name,tel,url,adress1,address2,city,zip,focus,summary
0,4,"(-74.01375579519738, 40.703816216918035)",Alexander Hamilton U.S. Custom House,(212) 514-3700,http://www.oldnycustomhouse.gov/,1 Bowling Grn,NaN,New York,10004.0,NaN,NaN
1,8,"(-74.0396848374904, 40.69905626244391)",American Immigration History Center,(212) 363-3200,http://www.ellisisland.org/,Ellis Island,NaN,New York,0.0,NaN,NaN
2,9,"(-73.99392890421056, 40.738015384101416)",American Jewish Historical Society,(212) 294-6160,http://www.ajhs.org/,15 W. 16th St.,NaN,New York,10011.0,NaN,NaN
3,12,"(-73.99388579658319, 40.73807723317813)",American Sephardi Federation / Sephardic House,(212) 294-8350,http://www.americansephardifederation.org/,15 W. 16th St.,NaN,New York,10011.0,NaN,NaN
4,20,"(-73.94654590494326, 40.83358228246046)",Audubon Terrace,NaN,http://www.washington-heights.us/history/archi...,Broadway at 155 Street,NaN,New York,10032.0,NaN,NaN
5,30,"(-73.87974874003561, 40.87823644134894)",Bronx Historical Society & Musem,(718) 881-8900,http://www.bronxhistoricalsociety.org/,3309 Bainbridge Ave,NaN,Bronx,10467.0,NaN,NaN
6,34,"(-73.94401601383748, 40.67450699609931)",Brooklyn Children's Museum,(718) 735-4400,http://www.brooklynkids.org/,145 Brooklyn Avenue,NaN,Brooklyn,11213.0,NaN,NaN
7,44,"(-74.00731553374261, 40.74789760206738)",Chelsea Art Museum,(212) 255-0719,http://www.chelseaartmuseum.org/,556 West 22nd Street,NaN,New York,10011.0,NaN,NaN
8,46,"(-73.97727391968168, 40.78587092106756)",Children's Museum of Manhattan,(212) 721-1234,http://www.cmom.org/,212 West 83rd Street,NaN,New York,10024.0,NaN,NaN
9,53,"(-73.93173408706043, 40.86492363574811)",Cloisters,(212) 923-3700,http://www.metmuseum.org/cloisters/,99 Margaret Corbin Drive,NaN,New York,10040.0,NaN,NaN


##### save current final dataframe into csv or xlsx for Tableau to read, comment out when done

In [140]:
# df_final.to_csv('final_museum.csv')